# Program HMM

In [1]:
import numpy as np
import pandas as pd
import math

### Ввод количества состояний, возможных сигналов и длины наблюдаемой последовательности

In [2]:
M = int(input('Введите количество состояний:'))
K = int(input('Введите количество излучений:'))
L = int(input('Введите длину последовательности:'))

Введите количество состояний:6
Введите количество излучений:8
Введите длину последовательности:16


### Генерируем случайные матрицы и сохраняем в .csv

In [3]:
transition = pd.DataFrame(np.random.randint(0,10,size=(M, M))).add_prefix('p_').T.add_prefix('p_').T
emission = pd.DataFrame(np.random.randint(0,10,size=(M, K))).add_prefix('e_').T.add_prefix('p_').T
begin_distribution = pd.DataFrame(np.random.randint(1,10,size=(M, 1))).T.add_prefix('p_').T
transition = transition.div(transition.sum(axis=1),axis=0)
emission = emission.div(emission.sum(axis=1),axis=0)
begin_distribution = begin_distribution.div(begin_distribution.sum(axis=0),axis=1)
transition.to_csv('transition.csv',sep='\t')
emission.to_csv('emission.csv',sep='\t')
begin_distribution.to_csv('begin_distribution.csv',sep='\t')

### Считываем полученные матрицы

In [4]:
transition1 = pd.read_csv('transition.csv',sep='\t',index_col=0)
emission1 = pd.read_csv('emission.csv',sep='\t',index_col=0)
begin_distribution1 = pd.read_csv('begin_distribution.csv',sep='\t',index_col=0)

### Cгенерируем последовательность длиной $L$ событий $e_k$

In [5]:
def random_choice(distribution,p):
    s = np.random.random()*distribution[p].sum()
    for i in range(len(distribution)):
        s = s - distribution[p]['p_'+str(i)]
        if s < 0:
            return 'p_'+str(i)
        
def random_choice2(distribution,p):
    s = np.random.random()*distribution[p].sum()
    for i in range(len(distribution)):
        s = s - distribution[p]['e_'+str(i)]
        if s < 0:
            return 'e_'+str(i)

def HMM_random_generator_path(begin_distribution1,transition1,L):
    path = []
    path.append(random_choice(begin_distribution1,'0'))
    for i in range(1,L):
        path.append(random_choice(pd.DataFrame(transition1.T[path[i-1]]),path[i-1]))
    return path

def HMM_random_generator_sequence(states,emission1,L):
    sequence = []
    for i in range(L):
        sequence.append(random_choice2(pd.DataFrame(emission1.T[states[i]]),states[i]))
    return sequence

In [6]:
simulated_path = HMM_random_generator_path(begin_distribution1,transition1,L)
sequence = HMM_random_generator_sequence(simulated_path,emission1,L)

simulated_path

['p_5',
 'p_2',
 'p_0',
 'p_5',
 'p_4',
 'p_4',
 'p_5',
 'p_0',
 'p_2',
 'p_0',
 'p_4',
 'p_5',
 'p_3',
 'p_1',
 'p_0',
 'p_2']

In [7]:
sequence

['e_6',
 'e_0',
 'e_7',
 'e_3',
 'e_4',
 'e_1',
 'e_1',
 'e_6',
 'e_3',
 'e_3',
 'e_4',
 'e_1',
 'e_1',
 'e_0',
 'e_3',
 'e_1']

### Прямой алгоритм

In [8]:
def forward_propagation(begin_distribution1,transition1,emission1,sequence):
    F = np.zeros((L,M))
    F[0] = begin_distribution1['0']*emission1[sequence[0]]
    for i in range(1,L):
        F[i] = F[i-1]@transition1*emission1[sequence[i]]
    return pd.DataFrame(F).add_prefix('p_').T

def P(data,L):
    return data[L-1].sum()

F = forward_propagation(begin_distribution1,transition1,emission1,sequence)

In [9]:
F

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
p_0,0.074074,0.000463,0.000548,0.000032,0.000000,1.914782e-07,2.204470e-08,1.264465e-08,1.386897e-09,2.299369e-10,0.000000e+00,9.567560e-13,1.058580e-13,6.801696e-15,4.175156e-15,1.455043e-16
p_1,0.017429,0.001430,0.000260,0.000019,0.000002,3.680385e-07,4.217931e-08,8.041452e-09,5.780150e-10,5.366771e-11,1.046984e-11,1.775223e-12,2.033549e-13,2.428025e-14,1.459897e-15,1.720410e-16
p_2,0.004630,0.003653,0.000000,0.000039,0.000004,2.244829e-07,4.480718e-08,3.763120e-09,1.039533e-09,1.435271e-10,1.623072e-11,1.095404e-12,2.174350e-13,1.815297e-14,1.881359e-15,4.391551e-16
p_3,0.018018,0.000000,0.000485,0.000023,0.000001,5.040463e-07,6.203975e-08,2.245017e-09,1.174495e-09,2.063821e-10,3.315043e-12,2.515497e-12,2.969112e-13,0.000000e+00,3.988956e-15,4.386584e-16
p_4,0.006173,0.002948,0.000328,0.000000,0.000010,7.949088e-07,9.214140e-08,1.385291e-08,0.000000e+00,0.000000e+00,4.943727e-11,3.794165e-12,4.464135e-13,3.345498e-14,0.000000e+00,4.989664e-16
p_5,0.030146,0.004712,0.000132,0.000094,0.000005,6.433003e-07,7.579510e-08,1.341548e-08,2.496741e-09,2.566597e-10,2.885498e-11,3.035504e-12,3.682366e-13,4.626224e-14,5.004270e-15,4.319888e-16


#### Для нашей последовательности посчитаем P(x)

In [10]:
print ('P(x)=',P(F,L))

P(x)= 2.1263140936025417e-15


### Обратный алгоритм

In [11]:
def backward_propagation(F,transition1,emission1,sequence1):
    F1 = pd.DataFrame(np.zeros((M,L)),index=transition1.index)
    F1[len(sequence)-1]=F[len(sequence)-1]
    for j in range(len(sequence)-2,-1,-1):
        for i in F1.index:
            p_sum = 0
            for k in F1.index:
                p_sum += F1[j+1][k]*transition1[k][i]*emission1[sequence1[j+1]][k]
            F1[j][i] = p_sum
    return F1

In [12]:
backward_propagation(F,transition1,emission1,sequence)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
p_0,7.125305e-30,3.410590e-29,4.594911e-28,6.130910e-27,4.260608e-26,3.517655e-25,2.524013e-24,1.775700e-23,9.518172e-23,1.266634e-21,8.946153e-21,6.511363e-20,7.353875e-19,6.185561e-18,5.925994e-17,1.455043e-16
p_1,5.485836e-30,5.637800e-29,5.320063e-28,5.196987e-27,4.423094e-26,3.571052e-25,2.211424e-24,1.613834e-23,1.101600e-22,1.075443e-21,9.184690e-21,7.232887e-20,5.703000e-19,6.240721e-18,5.722838e-17,1.720410e-16
p_2,4.642843e-30,6.560032e-29,5.862607e-28,4.775732e-27,3.843164e-26,3.157832e-25,2.639991e-24,1.739307e-23,1.214189e-22,9.859963e-22,8.002579e-21,6.453844e-20,4.841909e-19,6.931909e-18,5.104794e-17,4.391551e-16
p_3,5.551929e-30,5.651419e-29,5.068403e-28,5.387977e-27,3.895740e-26,3.129261e-25,2.631082e-24,1.552693e-23,1.049473e-22,1.114905e-21,8.089681e-21,6.391164e-20,5.805886e-19,6.138893e-18,4.898202e-17,4.386584e-16
p_4,6.011738e-30,5.670066e-29,3.860395e-28,6.201605e-27,3.650155e-26,2.883634e-25,2.705398e-24,1.441266e-23,8.003760e-23,1.282530e-21,7.645362e-21,5.889029e-20,6.233611e-19,5.254760e-18,4.462981e-17,4.989664e-16
p_5,2.971393e-30,7.818636e-29,8.261619e-28,2.829210e-27,4.129822e-26,3.342237e-25,2.531972e-24,1.620184e-23,1.708104e-22,5.885375e-22,8.345057e-21,7.446124e-20,3.309552e-19,8.136197e-18,4.980470e-17,4.319888e-16


### Алгоритм Витерби

In [13]:
def get_max(values):
    maxv = values[0]
    maxi = 0
    for ind, val in enumerate(values):
        if val>maxv:
            maxv = val
            maxi = ind
    return maxv,maxi

def viterbi_algorithm(transition1,emission1,sequence1):
    F = pd.DataFrame(np.zeros((M,L+1)),index=transition1.index)
    F[0]=begin_distribution1['0']
    c = pd.DataFrame(np.zeros((M,L+1)),index=transition1.index)
    for k in F.columns[1:]:
        for i in F.index:
            values = []
            for j in F.index:
                values.append(F[k-1][j]*transition1[i][j]*emission1[sequence1[k-1]][i])
            maxv, maxi = get_max(values)
            F[k][i] = maxv
            c[k][i] = maxi
    return F, c

def decoding(F,c):
    path = []
    idx = F[L].idxmax(axis=0)
    path = [idx]+path
    k = 'p_'+str(c[L][idx]).split('.')[0]
    path = [k] + path 
    for i in range(L-1,1,-1):
        k = 'p_'+ str(c[i][k]).split('.')[0]
        path = [k] + path
    return path

F, c = viterbi_algorithm(transition1,emission1,sequence)

In [14]:
F

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
p_0,0.296296,0.013717,0.000144,0.000038,9.633633e-07,0.000000e+00,1.148316e-09,6.977614e-11,1.453670e-11,7.328169e-13,5.665996e-14,0.000000e+00,4.732385e-17,2.464784e-18,5.705519e-20,1.849011e-20,2.347950e-22
p_1,0.111111,0.008715,0.000346,0.000015,4.857294e-07,3.377400e-08,2.552839e-09,1.181870e-10,8.754592e-12,2.431831e-13,9.320406e-15,1.080953e-15,9.017692e-17,4.174857e-18,1.932804e-19,4.295121e-21,2.939604e-22
p_2,0.037037,0.010101,0.000468,0.000000,1.562338e-06,4.784649e-08,1.808597e-09,1.098974e-10,3.634174e-12,5.946830e-13,2.997887e-14,1.931590e-15,7.453507e-17,3.882035e-18,1.283742e-19,7.131898e-21,8.824824e-22
p_3,0.333333,0.002966,0.000000,0.000037,5.356425e-07,2.069038e-08,3.910481e-09,2.376160e-10,3.143069e-12,7.130111e-13,5.512861e-14,3.970168e-16,1.611569e-16,8.393589e-18,0.000000e+00,1.799037e-20,7.995722e-22
p_4,0.037037,0.014815,0.000412,0.000015,0.000000e+00,1.093636e-07,5.063131e-09,2.344042e-10,1.302246e-11,0.000000e+00,0.000000e+00,3.863179e-15,1.788509e-16,8.280134e-18,2.300037e-19,0.000000e+00,7.003828e-22
p_5,0.185185,0.017540,0.000580,0.000007,2.583711e-06,6.976019e-08,4.238900e-09,1.962454e-10,1.271961e-11,9.834551e-13,4.957747e-14,2.874924e-15,1.497356e-16,6.932205e-18,3.209354e-19,1.426380e-20,7.818227e-22


In [15]:
c

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
p_0,0.0,5.0,5.0,5.0,3.0,0.0,5.0,5.0,5.0,5.0,5.0,0.0,5.0,5.0,5.0,5.0,5.0
p_1,0.0,3.0,4.0,4.0,3.0,5.0,4.0,4.0,4.0,4.0,3.0,3.0,4.0,4.0,4.0,4.0,3.0
p_2,0.0,0.0,0.0,0.0,0.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,5.0,5.0,5.0,5.0,0.0
p_3,0.0,5.0,0.0,5.0,1.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0.0,5.0,5.0
p_4,0.0,3.0,4.0,4.0,0.0,2.0,4.0,4.0,4.0,0.0,0.0,0.0,4.0,4.0,4.0,0.0,0.0
p_5,0.0,0.0,0.0,2.0,0.0,2.0,4.0,4.0,4.0,0.0,0.0,0.0,4.0,4.0,4.0,4.0,0.0


In [16]:
decoding(F,c)

['p_0',
 'p_5',
 'p_0',
 'p_2',
 'p_4',
 'p_4',
 'p_5',
 'p_0',
 'p_5',
 'p_0',
 'p_4',
 'p_4',
 'p_4',
 'p_5',
 'p_0',
 'p_2']

In [17]:
simulated_path

['p_5',
 'p_2',
 'p_0',
 'p_5',
 'p_4',
 'p_4',
 'p_5',
 'p_0',
 'p_2',
 'p_0',
 'p_4',
 'p_5',
 'p_3',
 'p_1',
 'p_0',
 'p_2']

## Нечестная игральная кость
### $ p_0 $ соответствует обычной игре в кости и $ p_1 $ для нечестной игры в кости
### $ e_0, ..., e_5 $ соответствует числам 1 ... 6 в выпавшей кости

In [20]:
M = int(input('Введите количество состояний:')) 
K = int(input('Введите количество излучений:')) 
L = int(input('Введите длину последовательности:'))

Введите количество состояний:2
Введите количество излучений:6
Введите длину последовательности:10


In [21]:
transition = pd.DataFrame(np.random.randint(0,10,size=(M, M))).add_prefix('p_').T.add_prefix('p_').T
emission = pd.DataFrame(np.random.randint(0,10,size=(M, K))).add_prefix('e_').T.add_prefix('p_').T
begin_distribution = pd.DataFrame(np.random.randint(1,10,size=(M, 1))).T.add_prefix('p_').T
transition = transition.div(transition.sum(axis=1),axis=0)
emission = emission.div(emission.sum(axis=1),axis=0)
begin_distribution = begin_distribution.div(begin_distribution.sum(axis=0),axis=1)
transition.to_csv('transition.csv',sep='\t')
emission.to_csv('emission.csv',sep='\t')
begin_distribution.to_csv('begin_distribution.csv',sep='\t')

In [22]:
transition1 = pd.read_csv('transition.csv',sep='\t',index_col=0)
emission1 = pd.read_csv('emission.csv',sep='\t',index_col=0)
begin_distribution1 = pd.read_csv('begin_distribution.csv',sep='\t',index_col=0)

In [23]:
## our data for casino
transition1['p_0']['p_0']=0.95
transition1['p_0']['p_1']=0.1
transition1['p_1']['p_0']=0.05
transition1['p_1']['p_1']=0.9
begin_distribution1['0']['p_0']=2/3
begin_distribution1['0']['p_1']=1/3
emission1['e_0']['p_0']=1/6
emission1['e_1']['p_0']=1/6
emission1['e_2']['p_0']=1/6
emission1['e_3']['p_0']=1/6
emission1['e_4']['p_0']=1/6
emission1['e_5']['p_0']=1/6
emission1['e_0']['p_1']=0.1
emission1['e_1']['p_1']=0.1
emission1['e_2']['p_1']=0.1
emission1['e_3']['p_1']=0.1
emission1['e_4']['p_1']=0.1
emission1['e_5']['p_1']=0.5

In [24]:
#simulate some path and sequence
simulated_path = HMM_random_generator_path(begin_distribution1,transition1,L)
sequence = HMM_random_generator_sequence(simulated_path,emission1,L)

In [25]:
F = forward_propagation(begin_distribution1,transition1,emission1,sequence)
B = backward_propagation(F,transition1,emission1,sequence)

In [26]:
F, c = viterbi_algorithm(transition1,emission1,sequence)

In [27]:
decoding(F,c)

['p_0', 'p_0', 'p_0', 'p_0', 'p_0', 'p_0', 'p_0', 'p_0', 'p_0', 'p_0']

In [28]:
simulated_path

['p_0', 'p_0', 'p_0', 'p_0', 'p_0', 'p_0', 'p_0', 'p_0', 'p_0', 'p_0']